# IBM Smoke Test (Phase A) Notebook

This notebook reproduces the two-qubit Bell-state smoke test described in the experimental plan.
It gives you an interactive way to validate the IBM hardware path before running larger studies.

## Prerequisites

1. Install the Quartumse package in the same environment as this notebook (this repository already provides it).
2. Make sure Qiskit is installed and up to date.
3. Set the `QISKIT_IBM_TOKEN` environment variable to a valid IBM Quantum Platform API token **before** you run any hardware cells.
4. Ensure you have runtime minutes available on the `ibm_torino` backend.

> ⚠️ The notebook will raise an error if the IBM token is missing.

### Option: Set the IBM token in-notebook

If you did not export `QISKIT_IBM_TOKEN` when launching the notebook server you can do it here.
Remove the comment marker and paste your token string between quotes.

```
import os
os.environ["QISKIT_IBM_TOKEN"] = "paste-your-token"
```

In [ ]:
# Load environment variables from .env file (if it exists)
try:
    from dotenv import load_dotenv
    load_dotenv()
    print('✓ Environment variables loaded from .env file')
except ImportError:
    print('⚠️ python-dotenv not installed. Install with: pip install python-dotenv')
    print('   Alternatively, set environment variables manually before starting Jupyter.')
except Exception as e:
    print(f'⚠️ Could not load .env file: {e}')

# Verify IBM token is available
import os
if not os.environ.get('QISKIT_IBM_TOKEN') and not os.environ.get('QISKIT_RUNTIME_API_TOKEN'):
    raise EnvironmentError(
        'QISKIT_IBM_TOKEN or QISKIT_RUNTIME_API_TOKEN is not set.\n'
        'Options:\n'
        '  1. Create a .env file with: QISKIT_RUNTIME_API_TOKEN=your-token-here\n'
        '  2. Set environment variable before starting Jupyter\n'
        '  3. Uncomment and use the manual token cell below\n'
        'Get your token from: https://quantum.ibm.com'
    )

# Show which token variable was found
token_var = 'QISKIT_IBM_TOKEN' if os.environ.get('QISKIT_IBM_TOKEN') else 'QISKIT_RUNTIME_API_TOKEN'
print(f'✓ IBM token detected from {token_var} (value not shown for security)')

## Imports and helpers

The helpers mirror the standalone smoke-test script so results line up with the plan.

In [ ]:
from pathlib import Path
import numpy as np
from qiskit import QuantumCircuit, transpile

from quartumse.connectors import resolve_backend, is_ibm_runtime_backend, create_runtime_sampler
from quartumse.shadows.core import Observable
from quartumse import ShadowEstimator
from quartumse.shadows import ShadowConfig
from quartumse.shadows.config import ShadowVersion
from quartumse.reporting.manifest import MitigationConfig

In [ ]:
def bell_circuit():
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    return qc


def measure_in_basis(qc, pauli_str):
    circuit = qc.copy()
    for i, axis in enumerate(pauli_str):
        if axis == 'X':
            circuit.h(i)
        elif axis == 'Y':
            circuit.sdg(i)
            circuit.h(i)
    circuit.measure_all()
    return circuit


def parity_from_counts(counts, pauli_str, shots):
    def bit_value(bitstring, index):
        return int(bitstring[-(index + 1)])

    expectation = 0.0
    for bitstring, ct in counts.items():
        weight = ct / shots
        parity = 1.0
        for idx, axis in enumerate(pauli_str):
            if axis != 'I':
                parity *= 1 - 2 * bit_value(bitstring, idx)
        expectation += weight * parity
    return expectation

## Connect to IBM hardware

This resolves the IBM backend descriptor and prepares the output directory used by the smoke test.

In [ ]:
backend, snapshot = resolve_backend('ibm:ibm_torino')
print('Connected to backend:', backend.name)
print('Backend version snapshot:', snapshot)
Path('validation_data').mkdir(exist_ok=True)

## Run direct measurement baselines

These replicate the 250-shot ZZ/XX parity checks. The results feed into the comparison table at the end.

In [ ]:
observables = [Observable('ZZ', 1.0), Observable('XX', 1.0)]
direct_shots = {
    'ZZ': 250,
    'XX': 250,
}

direct_results = {}
qc = bell_circuit()

# Check if we need to use IBM Runtime Sampler primitives
sampler = None
if is_ibm_runtime_backend(backend):
    sampler = create_runtime_sampler(backend)

for obs in observables:
    pauli = obs.pauli_string
    shots = direct_shots[pauli]
    circuit = measure_in_basis(qc, pauli)
    compiled = transpile(circuit, backend)
    
    if sampler is not None:
        job = sampler.run([compiled], shots=shots)
        result = job.result()
        counts = result[0].data.meas.get_counts()
    else:
        job = backend.run(compiled, shots=shots)
        counts = job.result().get_counts()
    
    expectation = parity_from_counts(counts, pauli, shots)
    direct_results[pauli] = {
        'expectation': float(expectation),
        'shots': shots,
        'counts': counts,
    }
    print(f"[Direct] {pauli}: expectation={expectation:.3f}, shots={shots}")

## Classical shadows v0 baseline

Uses the default (noise-agnostic) shadow workflow with 500 random measurements.

In [ ]:
shadow_v0 = ShadowEstimator(
    backend='ibm:ibm_torino',
    shadow_config=ShadowConfig(
        version=ShadowVersion.V0_BASELINE,
        shadow_size=500,
        random_seed=42,
    ),
    data_dir='validation_data',
)

result_v0 = shadow_v0.estimate(qc, observables, save_manifest=True)
print('Shadow v0 manifest saved to:', result_v0.manifest_path)

## Classical shadows v1 with measurement error mitigation

This run matches the plan's MEM-assisted configuration using 4×128 calibration shots.

In [ ]:
mem_shots = 128
shadow_v1 = ShadowEstimator(
    backend='ibm:ibm_torino',
    shadow_config=ShadowConfig(
        version=ShadowVersion.V1_NOISE_AWARE,
        shadow_size=200,
        random_seed=43,
        apply_inverse_channel=True,
    ),
    mitigation_config=MitigationConfig(techniques=[], parameters={'mem_shots': mem_shots}),
    data_dir='validation_data',
)

result_v1 = shadow_v1.estimate(qc, observables, save_manifest=True)
print('Shadow v1 manifest saved to:', result_v1.manifest_path)

## Compare the results

The cell below prints expectation values and (when available) the 95% confidence intervals pulled from the estimator outputs.

In [ ]:
def format_ci(ci):
    if ci is None:
        return 'N/A'
    return f"[{ci[0]:.3f}, {ci[1]:.3f}]"

summary_rows = []
for obs in observables:
    pauli = obs.pauli_string
    direct = direct_results[pauli]['expectation']
    v0_stats = result_v0.observables[pauli]
    v1_stats = result_v1.observables[pauli]

    summary_rows.append({
        'Observable': pauli,
        'Direct expectation': f"{direct:.3f}",
        'Shadows v0 expectation': f"{v0_stats['expectation_value']:.3f}",
        'Shadows v0 CI95': format_ci(v0_stats.get('ci_95')),
        'Shadows v1 expectation': f"{v1_stats['expectation_value']:.3f}",
        'Shadows v1 CI95': format_ci(v1_stats.get('ci_95')),
    })

import pandas as pd
summary_df = pd.DataFrame(summary_rows)
summary_df

## Validation artifacts

The smoke test writes manifest files and raw shot data under `validation_data/`. Use the cell below to confirm that captures occurred.

In [ ]:
validation_root = Path('validation_data')
for path in sorted(validation_root.rglob('*')):
    if path.is_file():
        print(path)